In [1]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [2]:
%cd nmt-kn-ml/

/content/nmt-kn-ml


In [3]:
! chmod a+x /content/nmt-kn-ml/scripts/data_preparation.sh
! chmod a+x /content/nmt-kn-ml/scripts/split_dataset.sh

In [4]:
! /content/nmt-kn-ml/scripts/data_preparation.sh data.kn data.ml

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 19.81 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 3.05 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 9.90 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads:

In [5]:
! /content/nmt-kn-ml/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.ml

In [6]:
%cd ..

/content


In [7]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [8]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
!fairseq-preprocess --source-lang kn --target-lang ml \
--trainpref nmt-kn-ml/train/train \
--validpref nmt-kn-ml/dev/dev \
--testpref nmt-kn-ml/test/test \
--destdir nmt-kn-ml/tokenized.kn-ml \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-06 03:47:23 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 03:47:23 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-ml/tokenized.kn-ml', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [15]:
!fairseq-train /content/nmt-kn-ml/tokenized.kn-ml \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --max-epoch 25 \
  --wandb-project "BiLSTM model - kn to ml - 6 March 2022"

2022-03-06 03:51:26 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-06 03:51:28 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 03:51:30 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'BiLSTM model - kn to ml - 6 March 2022', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1

In [16]:
!fairseq-generate /content/nmt-kn-ml/tokenized.kn-ml \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-779	-1.0655817985534668	ഇതിന ് റെ അടിസ ് ഥാനത ് തില ് ‍ പ ് രവര ് ‍ ത ് തിക ് കുന ് നു .
P-779	-4.1718 -0.0007 -1.3477 -4.5459 -0.0004 -1.3829 -0.0006 -2.3505 -0.0000 -0.0023 -3.9955 -0.0001 -3.0822 -0.0000 -0.0087 -0.0347 -0.0001 -1.6363 -0.0046 -1.0779 -0.0000 -1.9026 -0.0262 -0.0023
S-729	ನಾವೆಲ ್ ಲಾ ಸಮಾಜವಾದಿ ಕುಟುಂಬದ ಸದಸ ್ ಯರು .
T-729	നാം എല ് ലാവരും കുടുംബത ് തിന ് ‍ റെ ഭാഗമാണ ് .
H-729	-0.9865428805351257	ഞങ ് ങള ് ‍ അടുത ് ത ബന ് ധങ ് ങള ് ‍ ഞങ ് ങള ് ‍ .
D-729	-0.9865428805351257	ഞങ ് ങള ് ‍ അടുത ് ത ബന ് ധങ ് ങള ് ‍ ഞങ ് ങള ് ‍ .
P-729	-0.9566 -0.0001 -1.5636 -0.0004 -0.0038 -5.2500 -0.0002 -0.9922 -2.8010 -0.0000 -1.7730 -0.0001 -1.0884 -0.0002 -0.0026 -3.8156 -0.0000 -0.6517 -0.0003 -0.0048 -2.7952 -0.0040
S-104	ಅಂದರೆ , ಈವರೆಗೆ 2 ಲಕ ್ ಷ ರೂ .
T-104	നിലവില ് ‍ 2 ലക ് ഷമാണ ് നികുതിയിളവ ് .
H-104	-1.0277167558670044	ലക ് ഷക ് കണക ് കിന ് ലക ് ഷണങ ് ങൾ .
D-104	-1.0277167558670044	ലക ് ഷക ് കണക ് കിന ് ലക ് ഷണങ ് ങൾ .
P-104	-5.1779

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
!cp /content/checkpoints/checkpoint_best.pt /content/drive/MyDrive/